In [1]:
#Import Dependencies

import pandas as pd
from bs4 import BeautifulSoup as bs
import requests
import numpy as np
from sqlalchemy import create_engine

Extract and Transform

In [2]:
#Import CSV file create dataframe

CFB_2019_file = "Resources/CFB2019.csv"
CFB_2019_df = pd.read_csv(CFB_2019_file)
CFB_2019_df.head()

,Team,Games,Win-Loss,Off Rank,Off Plays,Off Yards,Off Yards/Play,Off TDs,Off Yards per Game,Def Rank,...,Average Time of Possession per Game,Turnover Rank,Fumbles Recovered,Opponents Intercepted,Turnovers Gain,Fumbles Lost,Interceptions Thrown.y,Turnovers Lost,Turnover Margin,Avg Turnover Margin per Game
0,Air Force (Mountain West),13,11-2,51,881,5483,6.22,55,421.8,17,...,34:28:00,78,9,7,16,12,6,18,-2,-0.15
1,Akron (MAC),12,0-12,130,725,2918,4.02,14,243.2,83,...,26:29:00,128,6,5,11,12,14,26,-15,-1.25
2,Alabama (SEC),13,11-2,6,842,6640,7.89,76,510.8,20,...,29:59:00,3,11,17,28,4,6,10,18,1.38
3,Appalachian St. (Sun Belt),14,13-1,39,969,6064,6.26,65,433.1,26,...,31:19:00,7,10,14,24,3,6,9,15,1.07
4,Arizona (Pac-12),12,4-8,30,873,5281,6.05,42,440.1,120,...,27:24:00,89,4,11,15,6,12,18,-3,-0.25


In [12]:
#Select desired columns

clean_CFB_2019_df = CFB_2019_df[["Team", "Win-Loss", "Off Rank", "Off TDs", "Def Rank", "Pass Yards", "Turnover Rank"]]

#Split team column into school and conference

split_CFB_2019_df = clean_CFB_2019_df["Team"].str.split("(", n=2, expand=True)

split_CFB_2019_df = split_CFB_2019_df.rename(columns={0: "School", 1: "Conference"})

#Add School column and rename

clean_CFB_2019_df["Team"] = split_CFB_2019_df["School"].astype(str)
clean_CFB_2019_df = clean_CFB_2019_df.rename(columns={"Team": "School","Win-Loss": "2019 Win-Loss", 
                                                      "Off Rank": "2019 Off Rank", "Off TDs": "2019 Off TDs", 
                                                      "Def Rank": "2019 Def Rank", "Pass Yards": "2019 Pass Yards"})

S
clean_CFB_2019_df["School"] = clean_CFB_2019_df["School"].str.strip()
clean_CFB_2019_df

C:\Users\swill\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,School,2019 Win-Loss,2019 Off Rank,2019 Off TDs,2019 Def Rank,2019 Pass Yards,Turnover Rank
0,Air Force,11-2,51,55,17,1602,78
1,Akron,0-12,130,14,83,2347,128
2,Alabama,11-2,6,76,20,4449,3
3,Appalachian St.,13-1,39,65,26,2824,7
4,Arizona,4-8,30,42,120,3193,89
...,...,...,...,...,...,...,...
125,West Virginia,5-7,119,28,74,2984,103
126,Western Ky.,9-4,84,38,24,3371,100
127,Western Mich.,7-6,25,56,87,3128,19
128,Wisconsin,10-4,38,56,4,2802,61


In [34]:
#Scrape sports reference website for historical table 

url = "https://www.sports-reference.com/cfb/schools"

tables = pd.read_html(url)
tables

[    Unnamed: 0_level_0   Unnamed: 1_level_0 Unnamed: 2_level_0  \
                     Rk               School               From   
 0                    1            Air Force               1957   
 1                    2                Akron               1975   
 2                    3              Alabama               1902   
 3                    4  Alameda Coast Guard               1943   
 4                    5         Alcorn State               1977   
 ..                 ...                  ...                ...   
 321                294              Wofford               1914   
 322                295                  WPI               1888   
 323                296              Wyoming               1905   
 324                297               Xavier               1935   
 325                298                 Yale               1872   
 
     Unnamed: 3_level_0 Unnamed: 4_level_0 Overall                     Bowls  \
                     To                Yrs     

In [35]:
#Create historical dataframe

cfb_df = tables[0]
cfb_df.columns = ['Rank', 'School', 'From', 'To', 'Years Playing', 'Overall G', 'Overall W', 'Overall L', 'Overall T', 'Overall Win Pct',
                 'Bowls G', 'Bowls W', 'Bowls L', 'Bowls T', 'Bowls Win Pct', 'SRS', 'SOS', 'AP', 'CC', 'Notes']

cfb_df.head()

,Rank,School,From,To,Years Playing,Overall G,Overall W,Overall L,Overall T,Overall Win Pct,Bowls G,Bowls W,Bowls L,Bowls T,Bowls Win Pct,SRS,SOS,AP,CC,Notes
0,1,Air Force,1957,2020,64,734,395,327,12,.546,27,13,13,1,.500,1.17,-0.60,7,3,NaN
1,2,Akron,1975,2020,35,395,150,242,3,.384,3,1,2,0,.333,-11.91,-7.08,0,1,NaN
2,3,Alabama,1902,2020,117,1277,930,305,42,.745,71,42,26,3,.613,13.51,4.14,57,27,record adjusted to 901-314-41 by NCAA
3,4,Alameda Coast Guard,1943,1944,2,15,8,4,3,.633,0,0,0,0,NaN,-0.16,-5.13,0,0,NaN
4,5,Alcorn State,1977,1977,1,11,3,8,0,.273,0,0,0,0,NaN,-17.10,-9.19,0,0,NaN


In [36]:
#Clean Historic data frame by:
#selecting columns
clean_historic_cfb_df = cfb_df[["School", "Years Playing", "Overall Win Pct", "Bowls Win Pct"]]

clean_historic_cfb_df["School"] = clean_historic_cfb_df["School"].astype(str)

#replacing Bowls Win Pct NAN with 0
clean_historic_cfb_df["Bowls Win Pct"] = clean_historic_cfb_df["Bowls Win Pct"].replace(np.nan, ".000")

#Stripping white space from School for merging
clean_historic_cfb_df["School"] = clean_historic_cfb_df["School"].str.strip()

clean_historic_cfb_df

C:\Users\swill\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\swill\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\swill\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the cavea

,School,Years Playing,Overall Win Pct,Bowls Win Pct
0,Air Force,64,.546,.500
1,Akron,35,.384,.333
2,Alabama,117,.745,.613
3,Alameda Coast Guard,2,.633,.000
4,Alcorn State,1,.273,.000
...,...,...,...,...
321,Wofford,15,.297,.000
322,WPI,2,.318,.000
323,Wyoming,109,.477,.500
324,Xavier,16,.450,.000


In [37]:
#request data from bleacher report for the best nfl player from every football team
player_url = "https://bleacherreport.com/articles/1007941-the-best-nfl-player-from-every-college-football-team"

response = requests.get(player_url)

In [38]:
#Create BeautifulSoup object 
player_soup = bs(response.text, 'html.parser')
print(player_soup.prettify())

<!DOCTYPE html>
<html class="no-js" data-reactroot="" lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="DPR,Width,Viewport-Width" http-equiv="Accept-CH"/>
  <meta content="1c424580-0f86-4d9b-88b2-bc8c0d029d4c" name="aol-te-auth"/>
  <meta content="mu-6e4ce5cd-57f20d11-7c0ecee9-d55c79e2" name="blitz"/>
  <meta content="7A63840181953B2A5A1FEA25FB45A991" name="msvalidate.01"/>
  <meta content="NOODP,NOYDIR" name="robots"/>
  <meta content="+Ntj422Jc4V03qgBqLYbF3LMvrursV0X2btn2Zoqn9w=" name="verify-v1"/>
  <meta content="The bowl season is winding down, and it is time to start talking about the best NFL players from every FBS college football team..." name="description"/>
  <meta content="College Football, Rankings List, Partners, Philly Lists, National Lists and Rankings" name="keywords"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta content="135174055162" property="fb:app_id"/>
  <meta content="teamstream://article/bleacherreport.com/article

In [39]:
#Scrape player and school from web page

players = player_soup.find_all('h1')

In [40]:
#Loop through list to only get players and school

player_list= []

i = 2
 
for player in players:
    
    if i <=121:
    
        player = players[i]
        player_list.append(player.text)
         
        i += 1
    
    


In [41]:
#Create dataframe of best nfl player from every football team

player_df = pd.DataFrame(player_list)

#and clean the dataframe by splitting columns

school_player_df = player_df[0].str.split(":", n=2, expand=True)

school_player_df = school_player_df.rename(columns={0: "School", 1: "Best NFL Player"})

#and stripping white space

school_player_df["School"] = school_player_df["School"].str.strip()                                                   

school_player_df

,School,Best NFL Player
0,Air Force,Chad Hennings
1,Akron,Jason Taylor
2,Alabama,Don Hutson
3,UAB,Roddy White
4,Arizona,Tedy Bruschi
...,...,...
115,West Virginia,Sam Huff
116,Western Kentucky,N/A
117,Western Michigan,Greg Jennings
118,Wisconsin,Mike Webster


In [42]:
#Merge clean_CFB_2019_df and clean_historic_cfb_df

school_merge_df = pd.merge(clean_CFB_2019_df,clean_historic_cfb_df, on="School")
school_merge_df

,School,2019 Win-Loss,2019 Off Rank,2019 Off TDs,2019 Def Rank,2019 Pass Yards,Years Playing,Overall Win Pct,Bowls Win Pct
0,Air Force,11-2,51,55,17,1602,64,.546,.500
1,Akron,0-12,130,14,83,2347,35,.384,.333
2,Alabama,11-2,6,76,20,4449,117,.745,.613
3,Arizona,4-8,30,42,120,3193,87,.532,.475
4,Arkansas,2-10,111,28,110,2315,115,.579,.393
...,...,...,...,...,...,...,...,...,...
79,Wake Forest,8-5,15,49,84,3748,113,.411,.643
80,Washington,8-5,79,48,35,3164,104,.596,.487
81,West Virginia,5-7,119,28,74,2984,108,.596,.405
82,Wisconsin,10-4,38,56,4,2802,129,.586,.516


In [43]:
#Merge school_merge_df and school_player_df

final_df = pd.merge(school_merge_df,school_player_df, on="School")
final_df

,School,2019 Win-Loss,2019 Off Rank,2019 Off TDs,2019 Def Rank,2019 Pass Yards,Years Playing,Overall Win Pct,Bowls Win Pct,Best NFL Player
0,Air Force,11-2,51,55,17,1602,64,.546,.500,Chad Hennings
1,Akron,0-12,130,14,83,2347,35,.384,.333,Jason Taylor
2,Alabama,11-2,6,76,20,4449,117,.745,.613,Don Hutson
3,Arizona,4-8,30,42,120,3193,87,.532,.475,Tedy Bruschi
4,Arkansas,2-10,111,28,110,2315,115,.579,.393,Lance Alworth
...,...,...,...,...,...,...,...,...,...,...
71,Wake Forest,8-5,15,49,84,3748,113,.411,.643,Bill George
72,Washington,8-5,79,48,35,3164,104,.596,.487,Warren Moon
73,West Virginia,5-7,119,28,74,2984,108,.596,.405,Sam Huff
74,Wisconsin,10-4,38,56,4,2802,129,.586,.516,Mike Webster


Load

In [44]:
#Create connection to postgesql

connection_string = "postgres:postgres@localhost:5432/Final_CFB_db"
engine = create_engine(f'postgresql://{connection_string}')

In [45]:
#Find Table names in database

engine.table_names()

['cfb2019', 'historical', 'final', 'best_player']

In [46]:
#Load final_df to postgresql

final_df.to_sql(name='final', con=engine, if_exists='append', index=True)

In [47]:
#Load lean_CFB_2019_df to postgresql


clean_CFB_2019_df.to_sql(name='cfb2019', con=engine, if_exists='append', index=True)

In [48]:
#Load clean_historic_cfb_df to postgresql

clean_historic_cfb_df.to_sql(name='historical', con=engine, if_exists='append', index=True)

In [49]:
#Load school_player_df to postgresql

school_player_df.to_sql(name='best_player', con=engine, if_exists='append', index=True)